<a href="https://colab.research.google.com/github/daniel27a/missp/blob/main/pulp/Resistor_bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Wykonał:
##Daniel Sobczak
###215342 7AiR2

Notebook zawiera rozwiązanie problemu liniowego przy pomocy narzędzia Pulp.

Poniższa komórka kodu zawiera instrukcję instalującą dodatek Pulp do Notebooka.

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

>Poniższy problem liniowy dotyczy minimalizacji mocy rozproszonej w dwójniku rezystorowym przedstawionym na rysunku poniżej:

![](https://github.com/daniel27a/missp/blob/main/photos/ResistorBridge.png?raw=true)

>Zmiennymi deyzyjnymi są wartości prądów w każdej z gałęzi. Wielkości napięć na rezystorach oraz zakresy w jakich mogą zmieniać się prądy przedstawia tabela poniżej:

  n| 1 | 2  | 3  | 4  | 5 
---|---|--- |--- |--- |---
Un [V] | 6 | 10 |  4 | 7  | 3 
In [mA]| 4 | 2 | 2 | 2 | 4 
delta(I) [mA] | 1 | 1 | 1 |1 | 1 

>Zarówno zmienne prądowe jak i stałe napięciowe są typu zmiennoprzecinkowego.

>Znając powyższe wartości, przechodzimy do rozwiązywania problemu minimalizacyjnego. Naszym głównym równaniem będzie suma mocy wydzielonych na poszczególnych rezystorach. Wydzielona moc równa się iloczynowi prądu i napięcia na danym odbiorniku.

>Dodatkowym ograniczeniem w badanym przypadku, jest prądowe prawo Kirchhoffa mówiące o tym że suma prądów wchodzących do węzła musi się równać sumie prądów z niego wychodzących.

>Wynikiem programu są obliczone prądy zapewniające optymalne rozwiązanie problemu oraz całkowita moc rozproszenia osiągnięta przy obliczonych prądach.

>Dodatkowo, program oblicza wartości rezystancji na podstawie prawa Ohma, mówiącego, że rezystancja odbiornika równa się ilorazowi napięcia do niego przyłożonego i prądu przez niego przepływającego.

>Wynik programu:

```
Status of problem: Optimal
I1 = 0.003 A
I2 = 0.001 A
I3 = 0.002 A
I4 = 0.001 A
I5 = 0.003 A

Power loss: 0.052000000000000005 W

R 1 = 2.0 k ohm
R 2 = 10.0 k ohm
R 3 = 2.0 k ohm
R 4 = 7.0 k ohm
R 5 = 1.0 k ohm
```












In [2]:
"""
Link to description:
https://www.youtube.com/watch?v=yZuVfekIXJ0

"""

import numpy as np
from pulp import *

# Name and type of problem
prob = LpProblem("Minimalization od diffused power in resistor bridge",LpMinimize)

# The 5 variables of currents with up and down limits.
i1=LpVariable("I1", 0.003, 0.005, LpContinuous)
i2=LpVariable("I2", 0.001, 0.003, LpContinuous)
i3=LpVariable("I3", 0.001, 0.003, LpContinuous)
i4=LpVariable("I4", 0.001, 0.003, LpContinuous)
i5=LpVariable("I5", 0.003, 0.005, LpContinuous)
 
# Constants of voltages
U = (6.0, 10.0, 4.0, 7.0, 3.0)

# The main function for total power in the circuit
prob += U[0]*i1 + U[1]*i2 + U[2]*i3 + U[3]*i4 + U[4]*i5, "Total Power"

# Constraints resulting from Kirchhoff's current law
prob += i1-i3 == i4, "KI1" 
prob += i2+i3 == i5, "KI2"

prob.writeLP("ResistorBridge.lp")

# The solution of problem
prob.solve()
print("Status of problem:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue, "A")
print()

# Sum of power loss
print("Power loss:", value(prob.objective), "W\n")  

# Listing of resistances of resistors resulting from Ohm's law
for v, k in zip(U, prob.variables()):
  print("R",U.index(v)+1,"=",(v/k.varValue/1000),"k ohm") 

Status of problem: Optimal
I1 = 0.003 A
I2 = 0.001 A
I3 = 0.002 A
I4 = 0.001 A
I5 = 0.003 A

Power loss: 0.052000000000000005 W

R 1 = 2.0 k ohm
R 2 = 10.0 k ohm
R 3 = 2.0 k ohm
R 4 = 7.0 k ohm
R 5 = 1.0 k ohm


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
